# Smart Locker with Intruder Alerts 

This project aims to simulate a smart locker system with intruder alerts using Proteus, VSPE, and Python. The system detects authorized and unauthorized users through face recognition and sends email alert with intruder image if unauthorized access is attempted.

## Project Components:
1. **Proteus Simulation**: Simulates the smart locker hardware setup.


2. **VSPE (Virtual Serial Port Emulator)**: Configures serial communication on COM1, enabling data exchange between Proteus and Python.


3. **Python Code**: Processes serial data, performs face recognition using the DeepFace library, and sends email alerts when unauthorized access is detected.



## Setup Instructions:

1. **Run the Simulation in Proteus**
   - Start the Proteus simulation for the smart locker system. This simulation includes components for face detection and access control, which will interact with the Python script.
   - Ensure that the simulation is active before proceeding, as the code depends on the simulated environment to function correctly.
   



2. **Configure VSPE for COM1**
   - Use the Virtual Serial Port Emulator (VSPE) to configure COM1 for serial communication.
   - This setup links the Proteus simulation to the Python code, allowing the Python script to send and receive signals from the simulated locker system.
   
  
  

3. **Run the Python Code**
   - With Proteus and VSPE running, execute the Python code.
   - The code listens for signals from COM1, processes face recognition,control the access control through Proteus simulation and sends an email alert with intruder image if an unauthorized face is detected.

## Detailed Expanation of Code

## Importing necessary modules

Importing necessary modules for email sending, serial communication, file handling, and DeepFace

In [ ]:
import smtplib
import serial
import os
import json
import shutil
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from deepface import DeepFace

## Serial Communication Setup

Initializing serial communication on COM1 with a baud rate of 9600

In [ ]:
# Initialize serial communication on COM1 with a baud rate of 9600
s = serial.Serial('COM1', 9600)

## Authorized Target Images

Defining  a list of authorized target images for verification and initializing Detection flag to track if detection has occurred

In [ ]:
# List of authorized target images
target_images = ["nayana.jpg", "alan.jpg", "nebu.jpg"]

# Variable to track if detection occurred
Detection = False

### SMTP Server Configuration for Sending Email Alerts

The following section sets up the necessary details for sending email alerts via SMTP.


In [ ]:
# SMTP server and port details for sending email alerts
smtp_port = 587  # Standard secure SMTP port
smtp_server = "smtp.gmail.com"  # Google SMTP Server

# Sender and receiver email addresses
email_from = "smartlockersender@gmail.com"
email_to = "smartlockerreceiver@gmail.com"

# Application-specific password for sender's email
pswd = "**********" 

## Email Sending Function

This function sends an email with an image attachment in case of a theft alert.

In [ ]:
# Function to send email with an image attachment 
def send_emails(email_to):
    
     # Email body content
    body = """
    Theft Detected before Smart Locker
    Theft Image Attached!
    """
    
    # Create MIME object for email content
    msg = MIMEMultipart()
    msg['From'] = email_from
    msg['To'] = email_to
    msg['Subject'] = "Theft Detected!!!"
    
    # Attach the email body to the MIME object
    msg.attach(MIMEText(body, 'plain'))

    # Define the image filename(captured image) to attach
    filename = "image_selected.jpg"
    
    # Open the image file in binary mode
    attachment= open(filename, 'rb')  
   
    # Encode attachment as base64
    attachment_package = MIMEBase('application', 'octet-stream')
    attachment_package.set_payload((attachment).read())
    encoders.encode_base64(attachment_package)
    attachment_package.add_header('Content-Disposition', "attachment; filename= " + filename)
    msg.attach(attachment_package)
    
    # Convert message to string format
    text = msg.as_string()

    # Connect to the SMTP server
    print("Connecting to server...")
    TIE_server = smtplib.SMTP(smtp_server, smtp_port)
    TIE_server.starttls() # Start TLS for security
    TIE_server.login(email_from, pswd) # Log in to the email server
    print("Succesfully connected to server")
    print()

    # Send email to the specified recipient
    print(f"Sending email to: {email_to}...")
    TIE_server.sendmail(email_from, email_to, text)
    print(f"Email sent to: {email_to}")
    print()

    # Close the SMTP server connection
    TIE_server.quit()    

## Face Comparison Function

This function compares two images using the **DeepFace** library to verify if the person in the captured image matches any of the authorized faces. The function returns "Match" if a match is found, "Not Match" if the face is unauthorized and "Failed" if a face can't be detected in one of the images.

In [ ]:
# Function to compare captured image with target images using DeepFace
def compare_images(img1_path, img2_path):
    try:
        # Perform facial verification using DeepFace
        result = DeepFace.verify(
            img1_path=img1_path,
            img2_path=img2_path,
            model_name="Facenet",
        )
        Detection=True # Set detection flag to True if any face is detected
    except ValueError as e:
        # Handle error if face is not detected properly    
        print(f"Face not detected properly: {e}")
        Detection=False # Set detection flag to False
        
    if Detection:
        print(json.dumps(result, indent=2)) # Print verification result in JSON format
        if result['verified']:
            print("The person is not a Theft.")
            return "Match"
        else:
            return "Not Match"
    else:
        return "Failed"

## Smart Locker System Monitoring Loop

This section of the code runs in an infinite loop to continuously check for locker access requests. 

Waiting for Serial Signal: The code first waits to receive data via serial communication. If the signal 'a' is received, it indicates that the Raspberry Pi has captured an image.

Image Retrieval: The code then searches for the captured image in a specific directory. Once located, it copies the image to a predefined location (image_selected.jpg) for further processing.

Face Verification Process: The code iterates over each target image in the list of authorized images and calls compare_images to compare the captured image with each target image.

Based on the result of the comparison: 

- If the captured face matches an authorized user, the system sends 'b' (face detected) and 'c' (face matched) signals via UART   to the Proteus, indicating successful authentication and allowing access to the locker.

- If no face is detected (i.e., compare_images returns "Failed"), the system sends 'd' to indicate that the face detection         failed, prompting the user to try again.

Unauthorized Access Handling: If the captured face does not match any authorized user (i.e., compare_images returns "Not Match"), the system interprets this as an unauthorized access attempt. It responds by:

- Sending 'b' (face detected) and 'e' (potential theft detected) to the Raspberry Pi to activate the security alarm.

- Calling send_emails to notify the locker owner of the unauthorized attempt, attaching the captured image of the unauthorized     user to the email.


In [ ]:
# Infinite loop to keep checking for locker access requests            
while True:
      print("Smart Locker System")
      serial_data = s.read() # Read serial data

      # If signal indicating an image is captured is received
      if(serial_data == b'a'):
          # Define the root path where captured image is stored
          root_path = r"C:\Users\HP\AppData\Local\Temp\VSM Studio"
          image_file = None # Initialize variable to hold image file path

          # Search for the captured image in the specified directory
          for root, dirs, files in os.walk(root_path):
              for file in files:
                  if file == 'image00000.jpg':
                      image_file = os.path.join(root, file)
                      break
              if image_file:
                  break

          # Copy the captured image to a new location for comparison      
          if image_file:
              source_image = r"C:\Users\HP\image_selected.jpg"
              shutil.copy(image_file, source_image)
              
          # Compare the captured image with each target image
          for target_image in target_images:
              status=compare_images(source_image, target_image)
              if status=="Match":
                  s.write(b'b') # Send confirmation signal to indicate a face is detected
                  s.write(b'c') # Send confirmation signal to unlock locker
                  break
              if status=="Failed":
                  s.write(b'd') # Send signal indicating failure in detecting a face
                  break
             
          # If no match is found
          if status=="Not Match":            
              s.write(b'b') # Send confirmation signal to indicate a face is detected
              s.write(b'e') # Send signal indicating potential theft
              print("Theft detected sending image on mail")
              send_emails(email_to) # Call function to send email alert

        